In [ ]:
################################################################
# テキストデータから、類似性検索のためのベクトルDBを作成する
################################################################

# 必要なライブラリをインストール
# この環境にはインストール済み
# !pip install langchain tiktoken sentence_transformers faiss-gpu boto3

In [ ]:

# オブジェクトストレージにアクセスするための設定
# 使用するオブジェクトストレージに合わせて変更すること
# キーをベタ書きしてるので良い子はまねしないでね


accesskey = "LpkzPlUyAwDO2CQHE0XnW307GdqSJtms"　
secretkey = "bUewoPAJnoKI2FhzJPYQnnFkLG-v382z"
endpoint = "http://10.38.76.10"
bucket = "langchain-bucket"


In [ ]:
# テキストデータをチャンク化する
import tiktoken
from langchain.document_loaders import S3FileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# オブジェクトストレージからテキストデータをロード
filename = "nutanixbible.txt"

loader = S3FileLoader(
    bucket,
    filename,
    aws_access_key_id=accesskey ,
    aws_secret_access_key=secretkey,
    endpoint_url=endpoint
)

document = loader.load()

# チャンクに分割
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=20,
)

splitted_texts = text_splitter.split_documents(document)
print(f"チャンクの総数:{len(splitted_texts)}")
print(f"チャンクされた文章の確認:\n{splitted_texts[6]}")


In [ ]:

# Embeddingモデルを使ってベクトル化し、ベクトルDBを作成する
# https://huggingface.co/intfloat/multilingual-e5-large

import sentence_transformers
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# 文章からベクトルに変換するためのモデル
embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-Large')

# ベクトルデータベースを作成
vectorstore = FAISS.from_documents(splitted_texts, embeddings)

In [ ]:
# ベクトルデータベースをオブジェクトストレージに保存

# pickleでベクトルデータベースをオブジェクト化

import boto3
import pickle

obj = pickle.dumps(vectorstore)

# オブジェクトストレージに保存
s3_client = boto3.client( 
    "s3",
    aws_access_key_id=accesskey ,
    aws_secret_access_key=secretkey,
    endpoint_url=endpoint
)
key = "vectorstore.pkl"
s3_client.put_object(Body=obj, Bucket=bucket, Key=key)

In [ ]:
###################################################
# ベクトル検索

import time

question = "NutanixのAOSのアップグレードが失敗した場合の影響を教えてください。"

start = time.time()
# 質問に対して、データベース中の類似度上位3件を抽出。質問の文章はこの関数でベクトル化され利用される
docs = vectorstore.similarity_search(question, k=3)
elapsed_time = time.time() - start
print(f"処理時間[s]: {elapsed_time:.2f}")
for i in range(len(docs)):
    print(docs[i])